In [19]:
import pandas as pd
import gurobipy as gp
import math

In [20]:
article_data = pd.read_csv("./data/article.csv")
article_data['MAXIMUM_ORDER_QUANTITY_TU'].fillna(1000, inplace=True)

article_data = article_data[(article_data['TEMPERATURE_ZONE'] == 'frozen')]
article_data = article_data.head(10)
article_data

,ARTICLE_ID,TEMPERATURE_ZONE,CATEGORY_LEVEL_1,CATEGORY_LEVEL_2,VOLUME_M3_PER_CU,MEAN_SHELF_LIFE,CU_PER_TU,ORDERING_COST_FIXED,ORDERING_COST_PER_TU,SALES_MARGIN_PER_CU,CLEARING_COST_PER_CU,MINIMUM_ORDER_QUANTITY_TU,MAXIMUM_ORDER_QUANTITY_TU
9,468a73f3,frozen,Tiefkühl,Kartoffelprodukte,0.001932,1000,1,3.7,0.1,0.01,0.1,0,1000.0
14,3bd76e22,frozen,Tiefkühl,Pizza & Baguettes,0.001825,1000,22,18.9,2.9,0.46,1.9,0,1000.0
23,84293966,frozen,Tiefkühl,Fertiggerichte & Snacks,0.001452,1000,18,16.2,2.4,0.38,1.5,0,1000.0
60,13541e66,frozen,Tiefkühl,"Fleisch, Fisch & Veggie",0.001576,1000,1,4.1,0.2,0.03,0.1,0,1000.0
85,d8d111cd,frozen,Tiefkühl,Eis,0.001016,1000,17,15.1,2.2,0.35,1.4,0,1000.0
106,ad634451,frozen,Tiefkühl,Pizza & Baguettes,0.002704,1000,15,13.7,2.0,0.30,1.2,0,23.0
191,05ffd974,frozen,Tiefkühl,Fertiggerichte & Snacks,0.001122,1000,3,4.9,0.3,0.05,0.2,0,1000.0
200,64833c1f,frozen,Tiefkühl,Backwaren & Desserts,0.001559,1000,14,13.3,1.9,0.29,1.2,0,1000.0
210,2f821a2e,frozen,Tiefkühl,Eis,0.002045,1000,2,4.2,0.2,0.03,0.1,0,1000.0
220,5aaffbac,frozen,Tiefkühl,Eis,0.002184,1000,21,18.3,2.8,0.44,1.8,0,1000.0


In [21]:
def createParameterMatrix(data, columns):
    parameters = []
    for column in columns:
        parameters.append(data[column].to_list())
    parameters = list(map(list, zip(*parameters)))
    return parameters

In [22]:
articles = article_data['ARTICLE_ID'].to_list()

parameters = createParameterMatrix(
    article_data,
    [
        'TEMPERATURE_ZONE',
        'VOLUME_M3_PER_CU',
        'MEAN_SHELF_LIFE',
        'CU_PER_TU',
        'ORDERING_COST_FIXED',
        'ORDERING_COST_PER_TU',
        'CLEARING_COST_PER_CU',
        'MINIMUM_ORDER_QUANTITY_TU',
        'MAXIMUM_ORDER_QUANTITY_TU'
    ]
)
parameters_dict = dict(zip(articles, parameters))

In [23]:
items, category, volume_per_cu, shelf_life, cu_per_tu, ordering_cost_fixed, ordering_cost_per_tu, clearing_cost_per_cu,minimum_order_quantity_tu, maximum_order_quantity_tu = gp.multidict(parameters_dict)

In [24]:
# time_periods = ['2022-06-13','2022-06-14','2022-06-15','2022-06-16','2022-06-17','2022-06-18']
time_periods = ['2022-06-13']
forecast_data = pd.read_csv('./data/sales_7.csv')
forecast_data = forecast_data[forecast_data['ARTICLE_ID'].isin(articles)]
demand = dict.fromkeys(articles)
for item in demand.keys():
    demand[item] = dict.fromkeys(time_periods)
    for time_period in demand[item].keys():
        count = forecast_data.loc[(forecast_data['ARTICLE_ID'] == item) & (forecast_data['DATE'] == time_period)]['PICKING_QUANTITY_CU']
        if(len(count)==0):
            demand[item][time_period] = 0
        else:
            demand[item][time_period] = count.values[0]
for item in items:
    for time_period in demand[item]:
        demand[item][time_period] = math.ceil(demand[item][time_period]/cu_per_tu[item])

In [11]:
# model object
m = gp.Model()

# decision variables
orders = m.addVars(items, time_periods, vtype=gp.GRB.INTEGER, lb=0, name=items)

# objective function
ordering_cost_objective = gp.quicksum(ordering_cost_per_tu[item] * orders[item, time_period] for item in items for time_period in time_periods)
m.setObjective(ordering_cost_objective, sense=gp.GRB.MINIMIZE)

# constraints
for item in items:
    for time_period in time_periods:
        m.addConstr(orders[item, time_period] >= demand[item][time_period],name="demand constraint_" + str(time_period))

m.optimize()
orders

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: AMD Ryzen 7 5800U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 10 rows, 10 columns and 10 nonzeros
Model fingerprint: 0x55ee1c78
Variable types: 0 continuous, 10 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-01, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 6e+01]
Found heuristic solution: objective 33.9000000
Presolve removed 10 rows and 10 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 33.9 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.390000000000e+01, best bound 3.390000000000e+01, gap 0.0000%


{('468a73f3', '2022-06-13'): <gurobi.Var 468a73f3 (value 58.0)>,
 ('3bd76e22', '2022-06-13'): <gurobi.Var 3bd76e22 (value 1.0)>,
 ('84293966', '2022-06-13'): <gurobi.Var 84293966 (value 3.0)>,
 ('13541e66', '2022-06-13'): <gurobi.Var 13541e66 (value 13.0)>,
 ('d8d111cd', '2022-06-13'): <gurobi.Var d8d111cd (value 1.0)>,
 ('ad634451', '2022-06-13'): <gurobi.Var ad634451 (value 3.0)>,
 ('05ffd974', '2022-06-13'): <gurobi.Var 05ffd974 (value 11.0)>,
 ('64833c1f', '2022-06-13'): <gurobi.Var 64833c1f (value 1.0)>,
 ('2f821a2e', '2022-06-13'): <gurobi.Var 2f821a2e (value 10.0)>,
 ('5aaffbac', '2022-06-13'): <gurobi.Var 5aaffbac (value 0.0)>}